In [1]:
import requests
from bs4 import BeautifulSoup
import pdfplumber
import pandas as pd
import re
from io import BytesIO

In [ ]:
base_url = "https://www.comunidad.madrid/servicios/educacion/publicaciones-interes-universitario"
html = requests.get(base_url).text
soup = BeautifulSoup(html, "html.parser")

pdf_links = [
    a["href"] for a in soup.find_all("a", href=True)
    if "notas" in a["href"].lower() and a["href"].endswith(".pdf")
]

if not pdf_links:
    raise Exception("No se encontró el PDF de notas de corte en la página.")

pdf_url = pdf_links[0]
if not pdf_url.startswith("http"):
    pdf_url = "https://www.comunidad.madrid" + pdf_url

response = requests.get(pdf_url)
pdf_bytes = BytesIO(response.content)

filas = []
año = 2025
convocatoria = "ordinaria"

with pdfplumber.open(pdf_bytes) as pdf:
    universidad_actual = None
    for page in pdf.pages:
        texto = page.extract_text()
        if not texto:
            continue
        # regex para buscar el nombre de la universidad en el texto de la página
        uni_match = re.search(r"(UNIVERSIDAD\s+[A-ZÁÉÍÓÚÑ\s]+?)(?:CURSO|\n|$)", texto)
        if uni_match:
            universidad_actual = uni_match.group(1).title().strip()
        table = page.extract_table()
        if not table:
            continue
        headers = table[0]
        data_rows = table[1:]
        for row in data_rows:
            if not any(row):
                continue
            codigo, grado, *resto = row
            if not grado or grado.lower().startswith("código"):
                continue
            for i, nota in enumerate(resto):
                if nota and re.search(r"\d", nota):
                    grupo = f"Grupo {i//2 + 1}"
                    valor = re.search(r"[\d,]+", nota)
                    if valor:
                        filas.append({
                            "universidad": universidad_actual,
                            "grado": grado.strip(),
                            "grupo": grupo,
                            "nota": valor.group(0).replace(",", "."),
                            "año": año,
                            "convocatoria": convocatoria
                        })

df_notas = pd.DataFrame(filas)
df_notas.drop_duplicates(subset=["universidad", "grado", "grupo"], inplace=True)
df_notas.to_csv("notas_corte.csv", index=False, encoding="utf-8")

In [23]:

df = pd.read_csv("notas_corte.csv", header=None, encoding="utf-8")

def extraer_campus(texto):
    if pd.isna(texto):
        return None

    matches = re.findall(r'\(([^()]*)\)', texto)
    if not matches:
        return None

    last = matches[-1].strip()

    if re.match(r'^[A-ZÁÉÍÓÚ]', last) and not re.search(r'PARS', last):
        return last
    return None

df["grados"] = df[1].apply(extraer_campus)

df.to_csv("notas_campus.csv", index=False, encoding="utf-8")

